In [2]:
!pip install -q langchain_community langchain_groq langchain langchain-deepseek 

In [ ]:
!git clone https://github.com/tli9181991/NewsRiskClassificationwithGemma2.git

fatal: destination path 'NewsRiskClassificationwithGemma2' already exists and is not an empty directory.


In [1]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import pandas as pd 
import os, sys
# from google.colab import files
from langchain_deepseek import ChatDeepSeek

In [ ]:
llm = ChatGroq(model_name = 'deepseek-r1-distill-llama-70b', api_key = GOOGLE_API_KEY)

In [2]:
DEEPSEEK_API_KEY =  'sk-c186e8b046bb4dfab4b3d9acd24f9dfd'
llm = ChatDeepSeek(
    model="deepseek-reasoner",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key = DEEPSEEK_API_KEY
)

In [ ]:
prompt = PromptTemplate.from_template(
    """
    You are a professional assistant for financial risk management.
    You will be given an risk analysis of a news.
    \n\n{analysis}
    State the risks mentioned in the analysis.
    If there is no risk, say no risk.
    Descriptions and reasons are no needed.
    """
)

In [ ]:
analysis_chain = (
    prompt
    |llm
    |StrOutputParser()
)

In [6]:
root_DS = './data/DSR1labeled/'
root_GA = './data/GEM3labeled/'

In [ ]:
for file in os.listdir(root_DS):
    df = pd.read_csv(os.path.join(root_DS, file))
    analysis = df['analysis']
    results = []
    print_str = ' '
    print(f"total news: {len(df)}")
    for i, ana in enumerate(analysis):
        sys.stdout.flush()
        print_str = '\r' + ''.join([' ' for i in range(len(print_str))])
        sys.stdout.write(print_str)

        sys.stdout.flush()
        print_str = f'\rprocessing news {i}'
        sys.stdout.write(print_str)

        if 'No risk' in ana:
            result = 'No risk.'
        else:
            result = analysis_chain.invoke({'analysis': ana})
        results.append(result)
    df['risks'] = results
    break

total news: 193
processing news 46

KeyboardInterrupt: 

In [ ]:
df['analysis'].iloc[3]

" The news about Nvidia's stock hitting a new high suggests positive performance and growth but could also indicate investor optimism that might lead to increased risk exposure if not balanced by actual sustainability factors."

In [ ]:
df['risks'].iloc[1]

'- Competition within the AI sector\n- Market headwinds from Chinese AI companies \n'

In [ ]:
analysis = df['analysis']
analysis

,analysis
0,The article discusses the potential of humano...
1,The risk classified for this news belongs to ...
2,No risk.
3,The news about Nvidia's stock hitting a new h...
4,The news article involves a case against indi...
...,...
188,The risk classification for the news is: Mark...
189,No risk.
190,The news belongs to the **Geopolitical/Market...
191,No risk.


In [7]:
root_GA = './data/GEM3labeled/'

In [8]:
def llm_labling(headline, description):

    prompt = PromptTemplate.from_template(
        """
        You are a professional assistant for financial risk management.
        You will be given the headline and the description of a news.
        Analysis the risk of the news of the article reflecting.
        \n\n{headline}
        \n\n{description}
        Classify the risk of the news to the following risks.
        
        If there is no risk, say no risk.
        Do not add any descriptions and reasons in your answer.
        """
    )
    labeling_chain = (
        prompt
        |llm
        |StrOutputParser()
    )

    result = labeling_chain.invoke({'headline': headline, 'description':description})
    analysis = result.split('/think>\n')[-1]
    analysis = analysis.replace('\n', ' ')

    return analysis

In [9]:
out_root = './data/DSRONlabeled'
sc_root = './data/'

In [ ]:
for file in os.listdir(sc_root):
    if not file.endswith('.csv'):
        continue

    if os.path.isfile(os.path.join(out_root, file)):
        print(f"{file} - skipped")
        print("\n----------------------------------------------------------\n")
        continue

    df = pd.read_csv(os.path.join(sc_root, file))

    llama3p2_analysis = []
    print_str = ' '
    print(f"{file} - total news: {len(df)}")
    for i, (headline, description) in enumerate(zip(df['headline'], df['description'])):
        sys.stdout.flush()
        print_str = '\r' + ''.join([' ' for i in range(len(print_str))])
        sys.stdout.write(print_str)

        sys.stdout.flush()
        print_str = f'\r{i} processing: {headline}'
        sys.stdout.write(print_str)

        inspected_risk = llm_labling(headline, description)

        llama3p2_analysis.append(inspected_risk)
    df['analysis'] = llama3p2_analysis
    df.to_csv(os.path.join(out_root, file))
    # files.download(os.path.join(out_root, file))
    print("\n----------------------------------------------------------\n")

yf_news_AAPL.csv - skipped

----------------------------------------------------------

yf_news_AMZN.csv - total news: 198
3 processing: This Is The Ultimate Warren Buffett Stock: Is It A Buy After CEO News, These Portfolio Moves?     

In [ ]:
df[['headline', 'analysis']]

,headline,analysis,llama3p2_analysis
0,The ‘next wave of AI growth’ could come from h...,Market Risk,Market Risk Technological Risk Operational Risk
1,Nvidia rally persists: What worries this analy...,Geopolitical Risk Competition Risk,Operational Risk Competition Risk Market Risk
2,"Tech trade check-in: Nvidia record high, Micro...",No risk.,"Market Risk, Financial Risk, Economic Risk"
3,Nvidia stock notches fresh record high in rema...,No risk.,Market Risk No Credit Risk No Liquidity Risk N...
4,Singapore case against three on AI chip fraud ...,Reputational Risk Legal Risk Regulatory Risk F...,1. Market risk 2. Credit risk 3. Operational r...
...,...,...,...
188,Nvidia Director Stevens Sells $88 Million in S...,No risk.,Market Risk (Sell-Off) Insider Trading Risk (E...
189,AMD Shares Rise as Analysts Predict Massive GP...,No risk.,"Market Risk, Equity Risk, Industry Risk"
190,"Nvidia, Broadcom Lead Market Rally as Middle E...",No risk.,Market Volatility Risk Geopolitical Risk Oil P...
191,Arm Reaches For New Highs After Laying Groundw...,Market Risk Operational Risk Financial Risk,Financial Risk Market Risk Operational Risk


In [ ]:
!pip install -q langchain_community langchain_groq langchain langchain-deepseek 

In [4]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import pandas as pd 
import os, sys
# from google.colab import files
from langchain_deepseek import ChatDeepSeek

In [5]:
DEEPSEEK_API_KEY =  'sk-c186e8b046bb4dfab4b3d9acd24f9dfd'
llm = ChatDeepSeek(
    model="deepseek-reasoner",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key = DEEPSEEK_API_KEY
)

In [44]:
def llm_labling(headline, description, ticker):

    prompt = PromptTemplate.from_template(
        """
        You are a professional in financial risk management.
        Given the headline and news description, analyze the event and rate each major risk category from -10 to +10 on the stock {ticker}:
        Only return in comma seperated format and skip all explainations.
        \n\n{headline}
        \n\n{description}
        -10 = Severe negative risk
        
        0 = No material impact
        
        +10 = Significant positive impact/opportunity
        
        Use the risk tree below. Focus on material relevance—if a risk does not apply, rate it 0.
        
        Risk Categories
        Market Risk
        – Interest Rate, Currency, Commodity, Volatility, Liquidity
        
        Financial Risk
        – Equity Valuation, Leverage, Cash Flow, Capital Structure
        
        Credit Risk
        – Credit Downgrade, Concentration, Bankruptcy, Counterparty
        
        Regulatory & Compliance Risk
        – Legal, AML, Regulatory Breach, KYC
        
        Political Risk
        – Geopolitical, Trade Sanctions, Instability
        
        Technology Risk
        – Cybersecurity, Outage, Data Breach
        
        Environmental Risk
        – Climate, Carbon Regulation, Natural Disasters
        
        Operational Risk
        – Process Failure, Human Error, Vendor
        
        Strategic Risk
        – Industry, Competition, M&A, Innovation
        
        Reputational Risk
        – Data Breach, ESG, Media/PR Crisis
        """
    )
    labeling_chain = (
        prompt
        |llm
        |StrOutputParser()
    )

    result = labeling_chain.invoke({'headline': headline, 'description':description, 'ticker':ticker})
    # analysis = result.split('/think>\n')[-1]
    # analysis = analysis.replace('\n', ' ')

    return result

In [45]:
out_root = './data/DSRONlabeledV2'
sc_root = './data/'

In [46]:
os.makedirs(out_root, exist_ok=True)

In [ ]:
processed = 0
for file in os.listdir(sc_root):
    if not file.endswith('.csv'):
        continue

    if os.path.isfile(os.path.join(out_root, file)):
        print(f"{file} - skipped")
        print("\n----------------------------------------------------------\n")
        continue

    df = pd.read_csv(os.path.join(sc_root, file))

    analysis = []
    print_str = ' '
    print(f"{file} - total news: {len(df)}")
    ticker = file.split('.')[0].split('news_')[-1]
    for i, (headline, description) in enumerate(zip(df['headline'], df['description'])):
        sys.stdout.flush()
        print_str = '\r' + ''.join([' ' for i in range(len(print_str))])
        sys.stdout.write(print_str)

        sys.stdout.flush()
        print_str = f'\r{i} processing: {headline}'
        sys.stdout.write(print_str)

        inspected_risk = llm_labling(headline, description, ticker)

        analysis.append(inspected_risk)
        if (i == 1):
            break

    df['analysis'] = analysis
    df.to_csv(os.path.join(out_root, file))
    break
    print("\n----------------------------------------------------------\n")

yf_news_AAPL.csv - total news: 198
0 processing: PCE data, consumer sentiment, F1 the Movie: What to Watch

In [43]:
file.split('.')[0].split('news_')[-1]

'AAPL'

In [29]:
df['description'][:10]

0    Asking for a Trend host Josh Lipton takes a lo...
1    Meta is on a quest to dominate the AI world. O...
2    Warren Buffett is widely regarded as one of th...
3    SCOTTSDALE, Ariz., June 26, 2025--FalconEye Ve...
4    Investment bank JPMorgan on Thursday cut its p...
5    Apple on Thursday announced a series of update...
6    Early buyers warm up the market, then things c...
7    Apple has reworked the terms and conditions fo...
8    Apple needs China, and Alibaba just helped unl...
9    AAPL's Services arm hits 1B+ paid subscribers ...
Name: description, dtype: object

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("./data/DSRONlabeledV2/yf_news_AAPL.csv", index_col=0,encoding='latin1')

In [7]:
json_str = df['responses'][0]
json_str

'```json\n{\n  "Strategic Risk": 1,\n  "Operational Risk": 0,\n  "Financial Risk": 0,\n  "Market Risk": 0,\n  "Technology Risk": 0,\n  "Credit Risk": 0,\n  "Legal Risk": 0,\n  "Political Risk": 0,\n  "Reputational Risk": 1,\n  "ESG Risk": 0\n}\n```'

In [15]:
json_str = json_str.replace("```", "")
json_str = json_str.replace("\n", "")
json_str = json_str.replace("json", "")
# eval_str = 'risk_dict = ' + json_str
risk_dict = eval(json_str)
risk_dict

{'Strategic Risk': 1,
 'Operational Risk': 0,
 'Financial Risk': 0,
 'Market Risk': 0,
 'Technology Risk': 0,
 'Credit Risk': 0,
 'Legal Risk': 0,
 'Political Risk': 0,
 'Reputational Risk': 1,
 'ESG Risk': 0}

In [16]:
def jsonstr2dict(json_str):
    json_str = json_str.replace("```", "")
    json_str = json_str.replace("\n", "")
    json_str = json_str.replace("json", "")

    return eval(json_str)